In [1]:
import pandas as pd
import numpy as np

In [2]:
# train_prompts = pd.read_csv('data/train_prompts.csv')
# train_essays = pd.read_csv('data/train_essays.csv')
train_drcat_01 = pd.read_csv('data/train_drcat_01.csv',usecols=['text', 'label','source'])
train_drcat_02 = pd.read_csv('data/train_drcat_02.csv',usecols=['text', 'label','source'])
train_drcat_03 = pd.read_csv('data/train_drcat_03.csv',usecols=['text', 'label','source'])
train_drcat_04 = pd.read_csv('data/train_drcat_04.csv',usecols=['text', 'label','source'])
data = pd.concat([train_drcat_01,train_drcat_02,train_drcat_03,train_drcat_04],axis=0)
data = data.drop_duplicates(subset='text') # this data is 3-fold data
data = data.loc[data.source!='persuade_corpus'] # will join in separately with score
data = data.loc[data.source!='radek_500'] # will join in separately with 200 GPT4 and kept as validation

In [3]:
persuade = pd.read_csv('data/persuade_2.0_human_scores_demo_id_github.csv',usecols=['full_text', 'holistic_essay_score'])
persuade = persuade.rename(columns={'full_text': 'text',
                                    'holistic_essay_score': 'score',
                                })
persuade['source'] = 'persuade_corpus'
persuade['label'] = 0

In [4]:
chatgpt = pd.read_csv('data/ai_generated_train_essays.csv',usecols=['text'])
gpt4 = pd.read_csv('data/ai_generated_train_essays_gpt-4.csv',usecols=['text'])
chatgpt['source'] = 'chatgpt'
chatgpt['label'] = 1
gpt4['source'] = 'gpt4'
gpt4['label'] = 1

In [5]:
data_all = pd.concat([data,persuade,chatgpt,gpt4],axis=0)

In [6]:
data_all.score = data_all.score.fillna(-1)

In [7]:
data_all.groupby(['score','source']).text.count()

score  source               
-1.0   chat_gpt_moth            2421
       chatgpt                   500
       darragh_claude_v6        1000
       darragh_claude_v7        1000
       falcon_180b_v1           1055
       gpt4                      200
       llama2_chat              2421
       llama_70b_v1             1172
       llammistral7binstruct    2421
       mistral7binstruct_v2     2421
       original_moth            2370
       train_essays             1378
 1.0   persuade_corpus          1028
 2.0   persuade_corpus          5699
 3.0   persuade_corpus          8368
 4.0   persuade_corpus          6731
 5.0   persuade_corpus          3297
 6.0   persuade_corpus           873
Name: text, dtype: int64

In [8]:
data_all.groupby(['label','source']).text.count()

label  source               
0      original_moth             2370
       persuade_corpus          25996
       train_essays              1375
1      chat_gpt_moth             2421
       chatgpt                    500
       darragh_claude_v6         1000
       darragh_claude_v7         1000
       falcon_180b_v1            1055
       gpt4                       200
       llama2_chat               2421
       llama_70b_v1              1172
       llammistral7binstruct     2421
       mistral7binstruct_v2      2421
       train_essays                 3
Name: text, dtype: int64

In [10]:
# split by topics
val = data_all.loc[data_all.source.isin(['train_essays','chatgpt','gpt4'])]
train = data_all.loc[~data_all.source.isin(['train_essays','chatgpt','gpt4'])]

In [8]:
val.to_csv('data/val_data.csv',index=False)
train.to_csv('data/train_data.csv',index=False)

In [9]:
val = pd.read_csv('data/val_data.csv')
train = pd.read_csv('data/train_data.csv')